# Simple ReAct Agent from Scratch

## Introduction 
In this notebook, we will `build a simple ReAct (Retrieval-Augmented Generation) agent from scratch using the OpenAI API`. The agent will be able to process user inputs, perform predefined actions, and provide answers based on the results of these actions. The contents of this revised Notebook are based on [A simple Python implementation of the ReAct pattern for LLMs](https://til.simonwillison.net/llms/python-react-pattern)

### Import Required Libraries

In [44]:
# Import required libraries
from dotenv import load_dotenv
import os

### Load Environment Variables
Load the environment variables from a .env file to access the OpenAI API key.

In [45]:
# Load environment variables from a .env file
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
print(f"Your OpenAI API Key: {openai_api_key}")

Your OpenAI API Key: sk-ftrE5R1SUdZBr8XxRi3rtI4F5g2i47i67FfUv9AXmjT3BlbkFJqODP02vVsRNjuVbRKIfFInuuWKeXKv_SqGGoIUNokA


### Import OpenAI Library and Set API Key

In [46]:
# Import OpenAI library and set API key
import openai
import re
import httpx
openai.api_key = openai_api_key

In [47]:
# Last snippet code that it works properly
#import openai
#import re
#import httpx
#import os
#from dotenv import load_dotenv, find_dotenv

# Load environment variables
#_ = load_dotenv(find_dotenv())  # This will find and load your .env file
#openai.api_key = os.environ['OPENAI_API_KEY']

### Test the OpenAI API
Test the OpenAI API with a simple completion query.

In [48]:
#client = OpenAI()

In [49]:
# Test the OpenAI API with a simple completion query
# Correct OpenAI API usage
chat_completion = openai.ChatCompletion.create(
#chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

print(chat_completion.choices[0].message.content)

Hello! How can I assist you today?


In [50]:
#chat_completion.choices[0].message.content

**Note**  The openai Python library doesn't expose a client object directly; instead, you interact with openai.ChatCompletion.create() for generating completions.

### Define the Agent Class

In this section, we define `a Python class called Agent` that will serve as the core component of our ReAct agent. The Agent class is designed to handle user inputs, process them using the OpenAI API, and generate appropriate responses. The class includes methods for initializing the agent with a system prompt, handling user messages, and executing API calls to retrieve responses. This structure allows the agent to maintain a conversation flow, store context, and interact with predefined actions.

In [51]:
# Define the Agent class
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        # Correct OpenAI API usage
        completion = openai.ChatCompletion.create(
        #completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

**Note**: Fixing the Issue Use the OpenAI Library Directly: Replace client.chat.completions.create with the proper OpenAI library method: openai.ChatCompletion.create.

### Define the System Prompt for the Agent
In this section, we define the system prompt that provides the initial context and instructions for the agent. The prompt outlines the `agent's operation loop, which includes phases such as Thought, Action, PAUSE, and Observation`. It also specifies the `available actions the agent can perform`, such as calculate and average_dog_weight, along with example sessions to illustrate how the agent should respond to user questions. This structured prompt helps guide the agent's behavior and ensures consistent and relevant interactions.

In [52]:
# Define the system prompt for the agent
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

### Define the Actions the Agent Can Perform

In this section, we define `the specific actions that the agent is capable of executing`. Each action is implemented as a function, such as calculate for performing mathematical calculations and average_dog_weight for returning the average weight of a specified dog breed. These functions enable the agent to handle various user queries by executing the appropriate action and returning the corresponding result. The known_actions dictionary maps action names to their respective functions, allowing the agent to dynamically select and execute actions based on user input.

In [53]:
# Define the actions the agent can perform
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

### Initialize the Agent

In this section, we initialize the Agent class with the system prompt defined earlier. This setup is crucial as it provides the initial context for the agent, enabling it to understand and respond to user inputs appropriately. By initializing the agent with the system prompt, we ensure that the agent starts with the necessary instructions and can maintain a coherent conversation flow.

In [54]:
# Initialize the agent with the system prompt
abot = Agent(prompt)

**Note** :The functionality of the code remains the same regardless of the variable name you choose. In the above code, `abot` is simply a variable name chosen to instantiate the Agent class with the provided prompt. You can rename abot to any other valid Python variable name that makes sense in your context (e.g.:  agent, my_agent, or chatbot instead). 

### Run the Agent with a Test Question

In [55]:
# Run the agent with a test question
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I should look up the average weight of a Toy Poodle using the average_dog_weight action.
Action: average_dog_weight: Toy Poodle
PAUSE


### Run the average_dog_weight Action Directly

In [56]:
# Run the average_dog_weight action directly
result = average_dog_weight("Toy Poodle")

# Display the Result
result

'a toy poodles average weight is 7 lbs'

#### Prepare the Next Prompt for the Agent

In [57]:
# Prepare the next prompt for the agent based on the observation
next_prompt = "Observation: {}".format(result)

#### Run the Agent with the Next Prompt

In [58]:
abot(next_prompt)

'Answer: A Toy Poodle weighs an average of 7 lbs.'

#### Display the Agent's Messages

In this section, we print the messages stored within the agent to inspect the conversation history. This includes all interactions between the user and the agent, such as user inputs and the agent's responses. Displaying these messages helps to understand how the agent processes and responds to queries, and it can be useful for debugging and improving the agent's performance.

In [59]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

#### Reinitialize the Agent

In this section, we reinitialize the Agent class to reset its state with the same system prompt. This step is useful for starting a new conversation or for resetting the agent's context. By reinitializing the agent, we ensure that it starts fresh without any previous messages or context, allowing for accurate and isolated testing of new interactions.

In [60]:
# Reinitialize the agent
abot = Agent(prompt)

#### Ask a New Question to the Agent

In [61]:
# Ask a new question to the agent
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

'Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to get the combined weight.\nAction: average_dog_weight: Border Collie\nPAUSE'

#### Prepare and Print the Next Prompt

In [62]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


#### Run the Agent with the Next Prompt

In [63]:
abot(next_prompt)

'Action: average_dog_weight: Scottish Terrier\nPAUSE'

#### Prepare and Print the Next Prompt for the Scottish Terrier

In [64]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


#### Run the Agent with the Next Prompt

In [65]:
abot(next_prompt)

'Action: calculate: 37 + 20\nPAUSE'

#### Prepare and Print the Next Prompt for Calculation

In [66]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


#### Run the Agent with the Next Prompt for Calculation

In [67]:
abot(next_prompt)

'Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.'

#### Add a Loop for the Agent

In this section, we add a loop to the agent's functionality to handle multiple iterations of user inputs and agent responses. This loop allows the agent to process a series of actions and observations in a structured manner. By defining a regular expression to match actions and creating a query function, we enable the agent to dynamically execute actions, observe results, and generate appropriate responses over several turns. This iterative process enhances the agent's capability to handle complex interactions and maintain context across multiple exchanges.

In [68]:
# Define a regular expression to match actions
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

<>:2: SyntaxWarning: invalid escape sequence '\w'
<>:2: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Michela\AppData\Local\Temp\ipykernel_4008\3505718801.py:2: SyntaxWarning: invalid escape sequence '\w'
  action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action


In [69]:
# Define a function to query the agent
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [70]:
# Test the query function
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add them together to find their combined weight.
Action: average_dog_weight: Border Collie
PAUSE
 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Action: average_dog_weight: Scottish Terrier
PAUSE
 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: Now that I have the average weights of both the Border Collie and the Scottish Terrier, I can calculate their combined weight by adding these two values together.
Action: calculate: 37 + 20
PAUSE
 -- running calculate 37 + 20
Observation: 57
Answer: The combined average weight of a Border Collie and a Scottish Terrier is 57 lbs.


## Conclusion 
In this notebook, we successfully built a simple ReAct agent from scratch using the OpenAI API. The agent can process user inputs, perform predefined actions, and provide answers based on the results of these actions. This demonstrates the potential of combining retrieval-augmented generation techniques with powerful language models to create intelligent agents.

#### Extra Practice! 

- Example 1    

Let's change the system agent of the prompt to ask to perform some calculation and returns the number using Python. To sum up, now the  available actions are:  
`calculate:  
e.g. calculate: 4 * 7 / 3  
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary`  

Fifteen * twenty five #   
query("Fifteen * twenty five")   
Thought: The action required is a calculation   
Action: calculate: 15 * 25   
PAUSE  
 -- running calculate 15 * 25  
Observation: 375  
Answer: Fifteen times twenty five equals 375.  

In [71]:
# Define the system prompt for the agent
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

Example session:

Question: How much does 15 * 25 equal?
Thought: I should calculate the value using the calculate action.
Action: calculate: 15 * 25
PAUSE

You will be called again with this:

Observation: 375

You then output:

Answer: 15 * 25 equals 375
""".strip()

# Define the Agent class (from the previous code)
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=self.messages
        )
        return completion.choices[0].message.content

# Define the actions the agent can perform
def calculate(what):
    return eval(what)

known_actions = {
    "calculate": calculate
}

# Initialize the agent with the system prompt
agent = Agent(prompt)

# Run the agent with a test question
result = agent("How much does 15 * 25 equal?")
print(result)

Thought: I should calculate the value using the calculate action.
Action: calculate: 15 * 25
PAUSE


In [72]:
# Run the calculation directly
result = calculate("15 * 25")

# Display the Result
result

375

In [73]:
# Prepare the next prompt for the agent based on the observation(calculation)
next_prompt = "Observation: {}".format(result)

In [74]:
#Print the next prompt
agent(next_prompt)

'Answer: 15 * 25 equals 375'

Test with another question related to a different calculation.

In [75]:
# Reinitialize the agent 
agent = Agent(prompt)

In [76]:
# Run the agent with a new question
result = agent("How much does 4 * 7 / 3 equal?")
print(result)

Thought: I should calculate the value using the calculate action.
Action: calculate: 4 * 7 / 3
PAUSE


In [77]:
# Prepare the next prompt for the agent based on the observation(calculation)
result = calculate("4 * 7 / 3")

# Display the Result
result

9.333333333333334

In [78]:
# Prepare the next prompt for the agent based on the observation(calculation)
next_prompt = "Observation: {}".format(result)

In [79]:
agent (next_prompt)

'Answer: 4 * 7 / 3 equals 9.333333333333334'